# AtticTreasures: Build a training library

In [1]:
# this part of AttacTreasures requieres several spotify playlists
# it gets a 

## Import libraries

In [2]:
#import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from random import randrange
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials
import os

## Add YOUR Spotity credentials

In [3]:
#Initialize SpotiPy with user credentials
# Remove mine when uploading

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="9f9d46aed10c480b9b68543a2d15e2c9",
    client_secret="0016f56c1cc641159dc9e19ba85bc682"))

## Functions:

In [4]:
#get a huge collection of song info from spotify:
#first create a collection of song names:
#get playlist, extranct song uris:
 #uri of a playlist with 100 songs from different artists:
ironhack_tunes="spotify:playlist:1kVI7R6OrKNN8Bf8GMJq9q"
shower_playlist="spotify:playlist:37i9dQZF1DWSqmBTGDYngZ"
eighties_to_oughts_playlist="spotify:playlist:6xnhcezDUVM4q6CHbX0nTS"
longest_playlist_ever="spotify:playlist:6yPiKpy7evrwvZodByKvM9"
long_playlist="spotify:playlist:0twlW4iDRhKXAJPopD8PP0"
classical="spotify:playlist:7qvZykTVPjvEX2LCcXoHog"
rap="spotify:playlist:37i9dQZF1DX0XUsuxWHRQd"
jazz="spotify:playlist:05Hd48jdQIz3s8WRrvGnzf"
traditional_chinese="spotify:playlist:3DTJZ0wPXLCUKM0R86fXAy"
traditional_african="spotify:playlist:6P911F6Yy5TszDzeTEmsVI"
traditional_jewish="spotify:playlist:3bvTqM2UeFVOh9vKEu3m1Y"
folk="spotify:playlist:37i9dQZF1DWYV7OOaGhoH0"

In [5]:
#starting again:
def get_info_from_playlist(playlist_name):

    '''
    get name of song, artists, and uri of the song in a df
    '''
    
    results = sp.user_playlist_tracks("spotify", playlist_name)
    items = results['items']
    song_uris=[]
    songs = []
    artists = []
    song_id=[]
    
    #the the first 100 songs
    for song_item in items:
             # print(["track"][0].keys())
            songs.append(song_item["track"]["name"])
            artists.append(song_item["track"]["artists"][0]["name"])
            song_uris.append(song_item["track"]["uri"])
            song_id.append(song_item["track"]["uri"][14:])
    
    #while there is a next, keep collecting songs
    while results['next']:
        results = sp.next(results)
        items.extend(results['items'])
        for song_item in items:
            songs.append(song_item["track"]["name"])
            artists.append(song_item["track"]["artists"][0]["name"])
            song_uris.append(song_item["track"]["uri"])
            song_id.append(song_item["track"]["uri"][14:])
        
    #create a dictionary after the loop:
    songs_dict={
    "song titles":songs,
    "artists":artists,
    "songs uri":song_uris,
    "song id":song_id
    }

    #Each key becomes a dataframe column name:
    library_df=pd.DataFrame(songs_dict)
    
    return library_df



def get_song_features(song_uri):
    '''
    input a individual song uri
    output a df with a single row,
    and the features in columns
    '''
    features_dict=sp.audio_features(song_uri)
    features_df=pd.DataFrame(features_dict)
    
    return features_df

def get_uris(playlist_name):
    '''
    input a playlist
    output a list of song uris
    the output can be feeded to the function get_song_features
    '''
    #include the while
    playlist = sp.user_playlist_tracks("spotify", playlist_name)
    items = playlist["items"]
    song_uris = []
    
    for song in items:
        song_uris.append(song["track"]["uri"][14:])
    return song_uris


def get_uris_all(playlist_name):
    #with next:
    '''
    input a playlist
    output a list of song uris
    the output can be feeded to the function get_song_features
    '''
    #include the while
    playlist = sp.user_playlist_tracks("spotify", playlist_name)
    items = playlist["items"]
    song_uris = []
    
    for song_item in items:
        song_uris.append(song_item["track"]["uri"][14:])
    
    #while there is a next, keep collecting songs
    while playlist['next']:
        playlist = sp.next(playlist)
        items.extend(playlist['items'])
        for song_item in items:
            song_uris.append(song_item["track"]["uri"][14:])
        
    return song_uris


## Main

In [6]:
# Build the super playlist:

#List of playlist to include:
list_of_playlists=[ironhack_tunes, shower_playlist, eighties_to_oughts_playlist,longest_playlist_ever, classical, rap,jazz, traditional_chinese, traditional_african, traditional_jewish, folk]

list_of_df=[]

#append the df with the info (song names, artist and song uri)
#to a list of df:
for l in list_of_playlists:
    list_of_df.append(get_info_from_playlist(l))

#concat the dfs in a big df:
df_total_pl=pd.concat(list_of_df)


#drop duplicates:
final_df=df_total_pl.drop_duplicates()
len(final_df)

#drop the rows when the "song_id" is longer than 22. Corrupted row
A=final_df[final_df['song id'].map(len) == 22]
# keep the rows that do not(~) contain ":"" if is more effective than the 22 filter
A=final_df[~final_df['song id'].str.contains(r':')]

#from the final_df I extract the column song uri and pass it to get_song features with a loop
song_uris_list=A["song id"].tolist()
list_of_df_names=[]
for uri in song_uris_list:
    #append each df (a single row each), to a list
    list_of_df_names.append(get_song_features(uri))
    
#concat all the dataframes, since the index of each df is 0, ignore index
df_total_features=pd.concat(list_of_df_names, ignore_index=True)

#merge the info df and the feature df using song uri:
df_complete=A.merge(df_total_features, left_on='songs uri', right_on='uri')

## Potential issues, keep an eye on this

In [7]:
#watch for this kind of strings in "song id column".
#if this happens, the whole row is corrupted, it must be dropped
#I dropped 81 out of 13756 (line 22 of 1.4 Main)
'''
Arcade+Fire:Funeral:Neighborhood+%231+%28Tunnels%29:288
The+Black+Keys:Turn+Blue:Bullet+In+the+Brain:256
The+Blacke+Keys:Turn+Blue:Fever:246
The+Black+Keys:Turn+Blue:Gotta+Get+Away:182
The+Black+Keys:Turn+Blue:In+Our+Prime:278
'''

'\nArcade+Fire:Funeral:Neighborhood+%231+%28Tunnels%29:288\nThe+Black+Keys:Turn+Blue:Bullet+In+the+Brain:256\nThe+Blacke+Keys:Turn+Blue:Fever:246\nThe+Black+Keys:Turn+Blue:Gotta+Get+Away:182\nThe+Black+Keys:Turn+Blue:In+Our+Prime:278\n'

## save files

In [12]:
df_total_features.to_csv('Data/total_features_2.csv')

In [13]:
A.to_csv('Data/total_playlist_info_2.csv')

In [14]:
df_complete.to_csv('Data/greatest_df_2.csv')

In [ ]:
df_complete.to_pickle("Data/greatest_df_2.pkl")